In [60]:
import cellpose
from macrohet import tools, dataio, visualise
import glob
import os
import zarr
import numpy as np
import napari
from tqdm.auto import tqdm
from skimage import io

In [13]:
!nvcc --version
!nvidia-smi

from cellpose import core, utils, io, models, metrics

use_GPU = core.use_gpu()
yn = ['NO', 'YES']
print(f'>>> GPU activated? {yn[use_GPU]}')

model = models.Cellpose(gpu=True, model_type='cyto')

def segment(img, ):
    masks, flows, styles, diams = model.eval(img, diameter=350, channels=[0,0],
                                             flow_threshold=None, cellprob_threshold=0)
    return masks

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Wed_Nov_22_10:17:15_PST_2023
Cuda compilation tools, release 12.3, V12.3.107
Build cuda_12.3.r12.3/compiler.33567101_0
Thu Feb 15 16:03:22 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX A6000               On  | 00000000:65:00.0  On |                  Off |
| 30%   47C    P8              36W / 30

In [4]:
expt_ID = 'ND0002'
location =  'SYNO' #'DATA' # 'NEMO' #
base_dir = f'/mnt/{location}/macrohet_{location.lower()}/{expt_ID}/'
metadata_fn = os.path.join(base_dir, 'acquisition/Images/Index.idx.xml')
metadata = dataio.read_harmony_metadata(metadata_fn)  
metadata_fn = glob.glob(os.path.join(base_dir, 'acquisition/Assaylayout/*.xml'))[0]
assay_layout = dataio.read_harmony_metadata(metadata_fn, assay_layout=True,)

Reading metadata XML file...


0it [00:00, ?it/s]

Extracting metadata complete!
Reading metadata XML file...
Extracting metadata complete!


In [6]:
assay_layout

Strain Compound Concentration ConcentrationEC
Row Column                                              
3   1         UNI     CTRL             0             EC0
    2         UNI     CTRL             0             EC0
    3          WT     CTRL             0             EC0
    4          WT     CTRL             0             EC0
    5          WT      PZA            60            EC50
    6          WT      PZA            60            EC50
    7          WT      RIF           0.1            EC50
    8          WT      RIF           0.1            EC50
    9          WT      INH          0.04            EC50
    10         WT      INH          0.04            EC50
    11         WT      BDQ          0.02            EC50
    12         WT      BDQ          0.02            EC50
4   3          WT     CTRL             0             EC0
    4          WT     CTRL             0             EC0
    5          WT      PZA           400            EC99
    6          WT      PZA           400            EC99
    7          WT      RIF             2            EC99
    8          WT      RIF             2            EC99
    9          WT      INH             2            EC99
    10         WT      INH             2            EC99
    11         WT      BDQ           2.5            EC99
    12         WT      BDQ           2.5            EC99
5   3         RD1     CTRL             0             EC0
    4         RD1     CTRL             0             EC0
    5         RD1      PZA            60            EC50
    6         RD1      PZA            60            EC50
    7         RD1      RIF           0.1            EC50
    8         RD1      RIF           0.1            EC50
    9         RD1      INH          0.04            EC50
    10        RD1      INH          0.04            EC50
    11        RD1      BDQ          0.02            EC50
    12        RD1      BDQ          0.02            EC50
6   3         RD1     CTRL             0             EC0
    4         RD1     CTRL             0             EC0
    5         RD1      PZA           400            EC99
    6         RD1      PZA           400            EC99
    7         RD1      RIF             2            EC99
    8         RD1      RIF             2            EC99
    9         RD1      INH             2            EC99
    10        RD1      INH             2            EC99
    11        RD1      BDQ           2.5            EC99
    12        RD1      BDQ           2.5            EC99

In [200]:
acq_ID = (3, 4)

In [208]:
zarr_group = zarr.open(f'/mnt/SYNO/macrohet_syno/{expt_ID}/acquisition/zarr/{acq_ID}.zarr', mode='r')

In [209]:
images = zarr_group.images

In [210]:
images.shape

(150, 2, 3, 6048, 6048)

In [211]:
img = np.max(images[0,1,:,...], axis = 0)

In [212]:
img.shape

(6048, 6048)

In [206]:
viewer = napari.Viewer()

viewer.add_image(img)

<Image layer 'img' at 0x7f8c05724280>

In [213]:
viewer.add_image(img)

<Image layer 'img [1]' at 0x7f8bfd9ce520>

In [34]:

model = models.Cellpose(gpu=True, model_type='cyto3')


masks, flows, styles, diams = model.eval(img,
                                         diameter = 350)
                                         # diameter=None, #350
                                         # channels=[0,0],
                                         # flow_threshold=None, 
                                         # cellprob_threshold=0)

In [207]:
viewer.add_labels(masks)

<Labels layer 'masks' at 0x7f8c07eb0c40>

In [36]:
from macrohet import notify
notify.send_sys_message()

# Try whole stack overnight

In [42]:
image_stack = np.max(images[:,1,:,...], axis = 1)

In [43]:
mask_stack = []
for img in tqdm(image_stack):
    masks, flows, styles, diams = model.eval(img,
                                             diameter = 350)
                                             # diameter=None, #350
                                             # channels=[0,0],
                                             # flow_threshold=None, 
                                             # cellprob_threshold=0)
    mask_stack.append(masks)
segmentation = np.stack(mask_stack, axis = 0)

  0%|          | 0/150 [00:00<?, ?it/s]

In [46]:
viewer.add_image(image_stack)
viewer.add_labels(segmentation)

<Labels layer 'segmentation' at 0x7f8c656df250>

In [47]:
import btrack

In [50]:
with btrack.io.HDF5FileHandler(os.path.join(base_dir, f'labels/macrohet_seg_model/{acq_ID}_cellposev3_seg.h5'), 
                                       'w', 
                                       obj_type='obj_type_1'
                                       ) as writer:
            writer.write_segmentation(segmentation)

[INFO][2024/02/16 10:33:00 am] Opening HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/macrohet_seg_model/(3, 4)_celllposev3_seg.h5...
[INFO][2024/02/16 10:33:53 am] Closing HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/macrohet_seg_model/(3, 4)_celllposev3_seg.h5


# test track first


### Load rfp images

In [68]:
gfp_image_stack = image_stack
rfp_image_stack = np.max(images[:,0,:,...], axis = 1)

In [69]:
Mtb_load_thresh = 480

In [71]:
# define thresholds
segment_size_thresh = 5000
Mtb_load_thresh = 480

# define tracking scale factor
scale_factor = 1/5.04

In [70]:
intensity_image = manual_mtb_thresh = np.where(rfp_image_stack >= Mtb_load_thresh, 1, 0)

### Simple localisation

In [72]:
 # define features to use for tracking 
features = [
  "area",
  "major_axis_length",
  "minor_axis_length",
  "orientation",
  "mean_intensity",
    ]

properties = tuple(features)

In [73]:
objects = btrack.utils.segmentation_to_objects(segmentation=segmentation,
                                                   intensity_image=intensity_image, 
                                                   properties=properties,
                                                   scale=(scale_factor,scale_factor),
                                                   use_weighted_centroid=False, 
                                                   )    

[INFO][2024/02/16 11:25:01 am] Localizing objects from segmentation...
INFO:btrack.io._localization:Localizing objects from segmentation...
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 150/150 [05:14<00:00,  2.10s/it]
[INFO][2024/02/16 11:30:16 am] Objects are of type: <class 'dict'>
INFO:btrack.io.utils:Objects are of type: <class 'dict'>
[INFO][2024/02/16 11:30:16 am] ...Found 38265 objects in 150 frames.
INFO:btrack.io._localization:...Found 38265 objects in 150 frames.


In [74]:
with btrack.io.HDF5FileHandler(os.path.join(base_dir, f'labels/macrohet_seg_model/{acq_ID}_cellposev3_objs.h5'), 
                                       'w', 
                                       obj_type='obj_type_1'
                                       ) as writer:
            writer.write_objects(objects)

[INFO][2024/02/16 11:30:16 am] Opening HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/macrohet_seg_model/(3, 4)_cellposev3_objs.h5...
INFO:btrack.io.hdf:Opening HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/macrohet_seg_model/(3, 4)_cellposev3_objs.h5...
[INFO][2024/02/16 11:30:17 am] Writing objects/obj_type_1
INFO:btrack.io.hdf:Writing objects/obj_type_1
[INFO][2024/02/16 11:30:17 am] Writing labels/obj_type_1
INFO:btrack.io.hdf:Writing labels/obj_type_1
[INFO][2024/02/16 11:30:17 am] Loading objects/obj_type_1 (38265, 5) (38265 filtered: None)
INFO:btrack.io.hdf:Loading objects/obj_type_1 (38265, 5) (38265 filtered: None)
[INFO][2024/02/16 11:30:18 am] Writing properties/obj_type_1/area (38265,)
INFO:btrack.io.hdf:Writing properties/obj_type_1/area (38265,)
[INFO][2024/02/16 11:30:18 am] Writing properties/obj_type_1/major_axis_length (38265,)
INFO:btrack.io.hdf:Writing properties/obj_type_1/major_axis_length (38265,)
[INFO][2024/02/16 11:30:18 am] Writing properties/obj_type_1/m

In [75]:
objects = [o for o in objects if o.properties['area'] > segment_size_thresh]


In [76]:
for obj in objects:
    obj.properties = ({"Infected": True} 
                        if obj.properties['mean_intensity'] > 0 # index 2 for manual mtb channel 
                        else {"Infected": False})
    obj.properties = ({"Mtb area px": obj.properties['mean_intensity']*obj.properties['area']})

In [106]:
config_fn = '/home/dayn/analysis/models/btrack/particle_config_pnassign.json'
search_radius = 20

# initialise a tracker session using a context manager
with btrack.BayesianTracker() as tracker:
    # configure the tracker using a config file
    tracker.configure(config_fn)
    # set max search radius
    tracker.max_search_radius = search_radius
    # define tracking method
    tracker.tracking_updates = ["MOTION", "VISUAL"]
    # redefine features so that both channels are included in track measurements
    tracker.features = list(objects[0].properties.keys())
    # append the objects to be tracked
    tracker.append(objects)
    # set the tracking volume
    tracker.volume=((0, segmentation.shape[-2]*scale_factor), (0, segmentation.shape[-1]*scale_factor))
    # track them (in interactive mode)
    tracker.track(step_size=25)
    # generate hypotheses and run the global optimizer
    tracker.optimize()
    # store the tracks
    tracks = tracker.tracks

[INFO][2024/02/16 02:12:16 pm] Loaded btrack: /home/dayn/analysis/btrack/btrack/libs/libtracker.so
INFO:btrack.libwrapper:Loaded btrack: /home/dayn/analysis/btrack/btrack/libs/libtracker.so
[INFO][2024/02/16 02:12:16 pm] Starting BayesianTracker session
INFO:btrack.core:Starting BayesianTracker session
[INFO][2024/02/16 02:12:16 pm] Loading configuration file: /home/dayn/analysis/models/btrack/particle_config_pnassign.json
INFO:btrack.config:Loading configuration file: /home/dayn/analysis/models/btrack/particle_config_pnassign.json
[INFO][2024/02/16 02:12:16 pm] Objects are of type: <class 'list'>
INFO:btrack.io.utils:Objects are of type: <class 'list'>
[INFO][2024/02/16 02:12:17 pm] Starting tracking... 
INFO:btrack.core:Starting tracking... 
[INFO][2024/02/16 02:12:17 pm] Update using: ['MOTION', 'VISUAL']
INFO:btrack.core:Update using: ['MOTION', 'VISUAL']
[INFO][2024/02/16 02:12:17 pm] Tracking objects in frames 0 to 24 (of 150)...
INFO:btrack.core:Tracking objects in frames 0 to 2

GLPK Integer Optimizer 5.0
16944 rows, 14040 columns, 19608 non-zeros
14040 integer variables, all of which are binary
Preprocessing...
8472 rows, 14040 columns, 19608 non-zeros
14040 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 8472
Solving LP relaxation...
GLPK Simplex Optimizer 5.0
8472 rows, 14040 columns, 19608 non-zeros
*     0: obj =   4.973911752e+04 inf =   0.000e+00 (3580)
Perturbing LP to avoid stalling [1839]...
Removing LP perturbation [3640]...
*  3640: obj =   2.538365859e+04 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Integer optimization begins...
Long-step dual simplex will be used
+  3640: mip =     not found yet >=              -inf        (1; 0)
+  3640: >>>>>   2.538365859e+04 >=   2.538365859e+04   0.0% (1; 0)
+  3640: mip =   2.538365859e+04 >=     tree is empty   0.0% (0; 1)
INTEGER OPTIMA

[INFO][2024/02/16 02:12:27 pm] Ending BayesianTracker session
INFO:btrack.core:Ending BayesianTracker session


In [78]:
with btrack.io.HDF5FileHandler(os.path.join(base_dir, f'labels/macrohet_seg_model/{acq_ID}_cellposev3_tracks.h5'), 
                                       'w', 
                                       obj_type='obj_type_1'
                                       ) as writer:
            writer.write_tracks(tracks)
    

[INFO][2024/02/16 11:30:28 am] Opening HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/macrohet_seg_model/(3, 4)_cellposev3_tracks.h5...
INFO:btrack.io.hdf:Opening HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/macrohet_seg_model/(3, 4)_cellposev3_tracks.h5...
[INFO][2024/02/16 11:30:29 am] Writing objects/obj_type_1
INFO:btrack.io.hdf:Writing objects/obj_type_1
[INFO][2024/02/16 11:30:29 am] Writing labels/obj_type_1
INFO:btrack.io.hdf:Writing labels/obj_type_1
[INFO][2024/02/16 11:30:29 am] Loading objects/obj_type_1 (26814, 5) (26814 filtered: None)
INFO:btrack.io.hdf:Loading objects/obj_type_1 (26814, 5) (26814 filtered: None)
[INFO][2024/02/16 11:30:29 am] Writing properties/obj_type_1/area (26814,)
INFO:btrack.io.hdf:Writing properties/obj_type_1/area (26814,)
[INFO][2024/02/16 11:30:29 am] Writing properties/obj_type_1/major_axis_length (26814,)
INFO:btrack.io.hdf:Writing properties/obj_type_1/major_axis_length (26814,)
[INFO][2024/02/16 11:30:29 am] Writing properties/obj_type

In [101]:
napari_tracks, _, _ = btrack.utils.tracks_to_napari(tracks, ndim=2)

In [102]:
viewer.add_tracks(napari_tracks, scale=(1/scale_factor,1/scale_factor))

<Tracks layer 'napari_tracks' at 0x7f8c15ca9d60>

### filter tracks

In [114]:
napari_tracks, _, _ = btrack.utils.tracks_to_napari([t for t in tracks if len(t) >= 70], ndim=2)

In [115]:
len([t for t in tracks if len(t) >= 75])

70

In [116]:
viewer.add_tracks(napari_tracks, scale=(1/scale_factor,1/scale_factor), name = 'filtered')

<Tracks layer 'filtered [8]' at 0x7f8c04e4aeb0>

# comparison with previous segmentation model

In [127]:
with btrack.io.HDF5FileHandler('/mnt/SYNO/macrohet_syno/ND0002/labels/macrohet_seg_model/(3, 4)_first_pass.h5', 
                               'r',
                               obj_type='obj_type_1') as reader:
    # tracks = reader.tracks
    objects_2 = reader.objects

[INFO][2024/02/16 02:45:54 pm] Opening HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/macrohet_seg_model/(3, 4)_first_pass.h5...
INFO:btrack.io.hdf:Opening HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/macrohet_seg_model/(3, 4)_first_pass.h5...
[INFO][2024/02/16 02:45:54 pm] Loading objects/obj_type_1 (50141, 5) (50141 filtered: None)
INFO:btrack.io.hdf:Loading objects/obj_type_1 (50141, 5) (50141 filtered: None)
[INFO][2024/02/16 02:45:55 pm] Closing HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/macrohet_seg_model/(3, 4)_first_pass.h5
INFO:btrack.io.hdf:Closing HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/macrohet_seg_model/(3, 4)_first_pass.h5


In [129]:
config_fn = '/home/dayn/analysis/models/btrack/particle_config_pnassign.json'
search_radius = 20

# initialise a tracker session using a context manager
with btrack.BayesianTracker() as tracker:
    # configure the tracker using a config file
    tracker.configure(config_fn)
    # set max search radius
    tracker.max_search_radius = search_radius
    # define tracking method
    tracker.tracking_updates = ["MOTION", "VISUAL"]
    # redefine features so that both channels are included in track measurements
    tracker.features = list(objects_2[0].properties.keys())
    # append the objects to be tracked
    tracker.append(objects_2)
    # set the tracking volume
    tracker.volume=((0, segmentation.shape[-2]*scale_factor), (0, segmentation.shape[-1]*scale_factor))
    # track them (in interactive mode)
    tracker.track(step_size=25)
    # generate hypotheses and run the global optimizer
    tracker.optimize()
    # store the tracks
    tracks = tracker.tracks

[INFO][2024/02/16 02:47:30 pm] Loaded btrack: /home/dayn/analysis/btrack/btrack/libs/libtracker.so
INFO:btrack.libwrapper:Loaded btrack: /home/dayn/analysis/btrack/btrack/libs/libtracker.so
[INFO][2024/02/16 02:47:30 pm] Starting BayesianTracker session
INFO:btrack.core:Starting BayesianTracker session
[INFO][2024/02/16 02:47:30 pm] Loading configuration file: /home/dayn/analysis/models/btrack/particle_config_pnassign.json
INFO:btrack.config:Loading configuration file: /home/dayn/analysis/models/btrack/particle_config_pnassign.json
[INFO][2024/02/16 02:47:30 pm] Objects are of type: <class 'list'>
INFO:btrack.io.utils:Objects are of type: <class 'list'>
[INFO][2024/02/16 02:47:31 pm] Starting tracking... 
INFO:btrack.core:Starting tracking... 
[INFO][2024/02/16 02:47:31 pm] Update using: ['MOTION', 'VISUAL']
INFO:btrack.core:Update using: ['MOTION', 'VISUAL']
[INFO][2024/02/16 02:47:31 pm] Tracking objects in frames 0 to 24 (of 150)...
INFO:btrack.core:Tracking objects in frames 0 to 2

GLPK Integer Optimizer 5.0
28504 rows, 24393 columns, 34534 non-zeros
24393 integer variables, all of which are binary
Preprocessing...
14252 rows, 24393 columns, 34534 non-zeros
24393 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 14252
Solving LP relaxation...
GLPK Simplex Optimizer 5.0
14252 rows, 24393 columns, 34534 non-zeros
*     0: obj =   8.367006095e+04 inf =   0.000e+00 (6978)
Perturbing LP to avoid stalling [2415]...
Removing LP perturbation [7046]...
*  7046: obj =   4.136481206e+04 inf =   0.000e+00 (0) 2
OPTIMAL LP SOLUTION FOUND
Integer optimization begins...
Long-step dual simplex will be used
+  7046: mip =     not found yet >=              -inf        (1; 0)
+  7046: >>>>>   4.136481206e+04 >=   4.136481206e+04   0.0% (1; 0)
+  7046: mip =   4.136481206e+04 >=     tree is empty   0.0% (0; 1)
INTEGER O

[INFO][2024/02/16 02:47:57 pm] Completed optimization with 5500 tracks
INFO:btrack.core:Completed optimization with 5500 tracks
[INFO][2024/02/16 02:47:57 pm] Ending BayesianTracker session
INFO:btrack.core:Ending BayesianTracker session


In [130]:
napari_tracks, _, _ = btrack.utils.tracks_to_napari(tracks, ndim=2)

In [131]:
viewer.add_tracks(napari_tracks, scale=(1/scale_factor,1/scale_factor))

<Tracks layer 'napari_tracks' at 0x7f8bf54c4940>

### filter tracks

In [132]:
napari_tracks, _, _ = btrack.utils.tracks_to_napari([t for t in tracks if len(t) >= 70], ndim=2)

In [133]:
len([t for t in tracks if len(t) >= 75])

149

In [134]:
viewer.add_tracks(napari_tracks, scale=(1/scale_factor,1/scale_factor), name = 'filtered')

<Tracks layer 'filtered [9]' at 0x7f8bfd2e9160>

# Check performance of other macroseg model on new acquisitions

In [169]:
assay_layout

Strain Compound Concentration ConcentrationEC
Row Column                                              
3   1         UNI     CTRL             0             EC0
    2         UNI     CTRL             0             EC0
    3          WT     CTRL             0             EC0
    4          WT     CTRL             0             EC0
    5          WT      PZA            60            EC50
    6          WT      PZA            60            EC50
    7          WT      RIF           0.1            EC50
    8          WT      RIF           0.1            EC50
    9          WT      INH          0.04            EC50
    10         WT      INH          0.04            EC50
    11         WT      BDQ          0.02            EC50
    12         WT      BDQ          0.02            EC50
4   3          WT     CTRL             0             EC0
    4          WT     CTRL             0             EC0
    5          WT      PZA           400            EC99
    6          WT      PZA           400            EC99
    7          WT      RIF             2            EC99
    8          WT      RIF             2            EC99
    9          WT      INH             2            EC99
    10         WT      INH             2            EC99
    11         WT      BDQ           2.5            EC99
    12         WT      BDQ           2.5            EC99
5   3         RD1     CTRL             0             EC0
    4         RD1     CTRL             0             EC0
    5         RD1      PZA            60            EC50
    6         RD1      PZA            60            EC50
    7         RD1      RIF           0.1            EC50
    8         RD1      RIF           0.1            EC50
    9         RD1      INH          0.04            EC50
    10        RD1      INH          0.04            EC50
    11        RD1      BDQ          0.02            EC50
    12        RD1      BDQ          0.02            EC50
6   3         RD1     CTRL             0             EC0
    4         RD1     CTRL             0             EC0
    5         RD1      PZA           400            EC99
    6         RD1      PZA           400            EC99
    7         RD1      RIF             2            EC99
    8         RD1      RIF             2            EC99
    9         RD1      INH             2            EC99
    10        RD1      INH             2            EC99
    11        RD1      BDQ           2.5            EC99
    12        RD1      BDQ           2.5            EC99

In [170]:
acq_ID = (4, 12)

In [171]:
zarr_group = zarr.open(f'/mnt/SYNO/macrohet_syno/{expt_ID}/acquisition/zarr/{acq_ID}.zarr', mode='r')

In [172]:
images = zarr_group.images

In [173]:
images.shape

(150, 2, 3, 6048, 6048)

In [177]:
%%time
images = np.max(images[:,1,:,...], axis = 1)

CPU times: user 39.2 s, sys: 58 s, total: 1min 37s
Wall time: 3min 21s


In [180]:
images.shape

(150, 6048, 6048)

In [182]:
with btrack.io.HDF5FileHandler(f'/mnt/SYNO/macrohet_syno/ND0002/labels/macrohet_seg_model/{acq_ID}_first_pass_seg_backup.h5', 
                               'r',
                               obj_type='obj_type_1') as reader:
    # tracks = reader.tracks
    segmentation_2 = reader.segmentation

[INFO][2024/02/16 03:35:39 pm] Opening HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/macrohet_seg_model/(4, 12)_first_pass_seg_backup.h5...
INFO:btrack.io.hdf:Opening HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/macrohet_seg_model/(4, 12)_first_pass_seg_backup.h5...
[INFO][2024/02/16 03:36:01 pm] Loading segmentation (150, 6048, 6048)
INFO:btrack.io.hdf:Loading segmentation (150, 6048, 6048)
[INFO][2024/02/16 03:36:01 pm] Closing HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/macrohet_seg_model/(4, 12)_first_pass_seg_backup.h5
INFO:btrack.io.hdf:Closing HDF file: /mnt/SYNO/macrohet_syno/ND0002/labels/macrohet_seg_model/(4, 12)_first_pass_seg_backup.h5


In [183]:
v_2 = napari.Viewer(title = f'{acq_ID}')

v_2.add_image(images)

<Image layer 'images' at 0x7f8c78d7bf40>

In [184]:
v_2.add_labels(segmentation_2)

<Labels layer 'segmentation_2' at 0x7f8c794c9ee0>

In [186]:
objects_2 =  btrack.utils.segmentation_to_objects(segmentation=segmentation_2,
                                                   intensity_image=images, 
                                                   properties=properties,
                                                   scale=(scale_factor,scale_factor),
                                                   use_weighted_centroid=False, 
                                                   )    

[INFO][2024/02/16 03:38:36 pm] Localizing objects from segmentation...
INFO:btrack.io._localization:Localizing objects from segmentation...
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 150/150 [03:43<00:00,  1.49s/it]
[INFO][2024/02/16 03:42:20 pm] Objects are of type: <class 'dict'>
INFO:btrack.io.utils:Objects are of type: <class 'dict'>
[INFO][2024/02/16 03:42:20 pm] ...Found 35489 objects in 150 frames.
INFO:btrack.io._localization:...Found 35489 objects in 150 frames.


In [187]:
objects_2 = [o for o in objects_2 if o.properties['area'] > segment_size_thresh]


In [188]:
config_fn = '/home/dayn/analysis/models/btrack/particle_config_pnassign.json'
search_radius = 20

# initialise a tracker session using a context manager
with btrack.BayesianTracker() as tracker:
    # configure the tracker using a config file
    tracker.configure(config_fn)
    # set max search radius
    tracker.max_search_radius = search_radius
    # define tracking method
    tracker.tracking_updates = ["MOTION", "VISUAL"]
    # redefine features so that both channels are included in track measurements
    tracker.features = list(objects_2[0].properties.keys())
    # append the objects to be tracked
    tracker.append(objects_2)
    # set the tracking volume
    tracker.volume=((0, segmentation.shape[-2]*scale_factor), (0, segmentation.shape[-1]*scale_factor))
    # track them (in interactive mode)
    tracker.track(step_size=25)
    # generate hypotheses and run the global optimizer
    tracker.optimize()
    # store the tracks
    tracks = tracker.tracks

[INFO][2024/02/16 03:43:11 pm] Loaded btrack: /home/dayn/analysis/btrack/btrack/libs/libtracker.so
INFO:btrack.libwrapper:Loaded btrack: /home/dayn/analysis/btrack/btrack/libs/libtracker.so
[INFO][2024/02/16 03:43:11 pm] Starting BayesianTracker session
INFO:btrack.core:Starting BayesianTracker session
[INFO][2024/02/16 03:43:11 pm] Loading configuration file: /home/dayn/analysis/models/btrack/particle_config_pnassign.json
INFO:btrack.config:Loading configuration file: /home/dayn/analysis/models/btrack/particle_config_pnassign.json
[INFO][2024/02/16 03:43:11 pm] Objects are of type: <class 'list'>
INFO:btrack.io.utils:Objects are of type: <class 'list'>
[INFO][2024/02/16 03:43:11 pm] Starting tracking... 
INFO:btrack.core:Starting tracking... 
[INFO][2024/02/16 03:43:11 pm] Update using: ['MOTION', 'VISUAL']
INFO:btrack.core:Update using: ['MOTION', 'VISUAL']
[INFO][2024/02/16 03:43:11 pm] Tracking objects in frames 0 to 24 (of 150)...
INFO:btrack.core:Tracking objects in frames 0 to 2

GLPK Integer Optimizer 5.0
10428 rows, 8726 columns, 12238 non-zeros
8726 integer variables, all of which are binary
Preprocessing...
5214 rows, 8726 columns, 12238 non-zeros
8726 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 5214
Solving LP relaxation...
GLPK Simplex Optimizer 5.0
5214 rows, 8726 columns, 12238 non-zeros
*     0: obj =   3.055618205e+04 inf =   0.000e+00 (2549)
Perturbing LP to avoid stalling [1243]...
Removing LP perturbation [2591]...
*  2591: obj =   1.429559724e+04 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Integer optimization begins...
Long-step dual simplex will be used
+  2591: mip =     not found yet >=              -inf        (1; 0)
+  2591: >>>>>   1.429559724e+04 >=   1.429559724e+04   0.0% (1; 0)
+  2591: mip =   1.429559724e+04 >=     tree is empty   0.0% (0; 1)
INTEGER OPTIMAL SOL

In [189]:
napari_tracks, _, _ = btrack.utils.tracks_to_napari(tracks, ndim=2)

In [190]:
v_2.add_tracks(napari_tracks, scale=(1/scale_factor,1/scale_factor))

<Tracks layer 'napari_tracks' at 0x7f8c78fc45e0>

### filter tracks

In [191]:
napari_tracks, _, _ = btrack.utils.tracks_to_napari([t for t in tracks if len(t) >= 70], ndim=2)

In [192]:
len([t for t in tracks if len(t) >= 75])

66

In [194]:
v_2.add_tracks(napari_tracks, scale=(1/scale_factor,1/scale_factor), name = 'filtered')

<Tracks layer 'filtered' at 0x7f8c15fe2fd0>

# Going back to cpv3 to chekc area measurements

In [198]:
objects[1]

,ID,x,y,z,t,dummy,states,label,area,major_axis_length,minor_axis_length,orientation,mean_intensity,Infected,Mtb area px
0,1,563.550037,13.155885,0.0,0,False,7,5,25162,234.099845,149.399704,1.500681,0.014228,True,358.0


In [199]:
viewer.add_points([0, 13*5.04, 563*5.04], size = 100)

<Points layer 'Points' at 0x7f8c05703190>

In [217]:
np.sum(viewer.layers['Labels'].data)

358

# First pass seg then second pass

In [150]:

model = models.Cellpose(gpu=True, model_type='cyto3')


INFO:cellpose.core:** TORCH CUDA version installed and working. **
INFO:cellpose.core:>>>> using GPU
INFO:cellpose.models:>> cyto3 << model set to be used
INFO:cellpose.models:>>>> model diam_mean =  30.000 (ROIs rescaled to this size during training)


In [166]:
masks, flows, styles, diams = model.eval(image_frame,
                                         diameter = 300,)
                                         # diameter=None, #350
                                         # channels=[0,0],
                                         # flow_threshold=None, 
                                         # cellprob_threshold=6)

INFO:cellpose.models:~~~ FINDING MASKS ~~~
INFO:cellpose.models:>>>> TOTAL TIME 70.33 sec


In [157]:
viewer.add_image(masks)

<Image layer 'masks [3]' at 0x7f8bfee78370>

In [167]:
viewer.add_labels(masks)

<Labels layer 'masks [6]' at 0x7f8bfc89d880>

In [168]:
pri

NameError: name 'pri' is not defined

# More training data

### Pick a frame

In [51]:
t = 0
image_frame = image_stack[t]
segmentation_frame = segmentation[t]

### Convert to semantic

In [52]:
gt_masks = tools.instance_to_semantic(segmentation_frame)

Iterating over segments:   0%|          | 0/228 [00:00<?, ?it/s]

In [55]:
acq_ID

(3, 4)

In [62]:
gt_image_fn = os.path.join(base_dir, f'labels/macrohet_seg_model/training/{acq_ID}_t{t}_gt_image.tif')
io.imsave(gt_image_fn, image_frame)

In [63]:
gt_mask_fn = os.path.join(base_dir, f'labels/macrohet_seg_model/training/{acq_ID}_t{t}_gt_masks.tif')

### Open new napari session

In [64]:
v = napari.Viewer()
v.add_image(image_frame, colormap='green', contrast_limits=[0,8000], blending = 'additive')
v.add_image(image_frame, colormap='green', contrast_limits=[0,4500], blending = 'additive', name = 'images_bright', visible = False)
visualise.add_napari_grid_overlay(viewer = v, N_rows_cols=5)

v.add_labels(gt_masks, color={1:'yellow'}, opacity=0.5, blending = 'additive')
# v.add_image(other_ch_image, colormap='magenta', contrast_limits = [0,4000], blending = 'additive', scale=[1/4, 1/4])
@v.bind_key("s", overwrite = True)
def save_masks(v):  
    v.layers['gt_masks'].save(gt_mask_fn.replace('semantic', 'backup'))
    print('Saved')
@v.bind_key("b", overwrite = True)
def show_bright_gt_image(v):  
    v.layers['images_bright'].visible = 1 if v.layers['ds_images_bright'].visible == 0 else 0

In [67]:
v.title = 'gt masks'